# Dependencies

In [1]:
# EXTERNAL LIBRARIES
import numpy as np 
import re

import torch
import torch.nn as nn 
import torch.optim as optim 

import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms

import os
import sys
import shutil

from torchvision.utils import save_image

In [2]:
# mister_ed
import recoloradv.mister_ed.loss_functions as lf 
import recoloradv.mister_ed.utils.pytorch_utils as utils
import recoloradv.mister_ed.utils.image_utils as img_utils
import recoloradv.mister_ed.cifar10.cifar_loader as cifar_loader
import recoloradv.mister_ed.cifar10.cifar_resnets as cifar_resnets
import recoloradv.mister_ed.adversarial_training as advtrain
import recoloradv.mister_ed.utils.checkpoints as checkpoints
import recoloradv.mister_ed.adversarial_perturbations as ap 
import recoloradv.mister_ed.adversarial_attacks as aa
import recoloradv.mister_ed.spatial_transformers as st
import recoloradv.mister_ed.config as config

# ReColorAdv
import recoloradv.perturbations as pt
import recoloradv.color_transformers as ct
import recoloradv.color_spaces as cs
from recoloradv import norms
from recoloradv.utils import load_pretrained_cifar10_model, get_attack_from_name

In [3]:
class HidePrint:
    def __enter__(self):
        self._temp_stdout = sys.stdout
        sys.stdout = open(os.devnull, 'w')

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout.close()
        sys.stdout = self._temp_stdout

In [4]:
def prepare_folders(level):
    if os.path.exists(f"../datasets/adv_{level}"):
        shutil.rmtree(f"../datasets/adv_{level}")
    for i in range(len(class_names)):
        os.makedirs(f"../datasets/adv_{level}/train/{i}")
        os.makedirs(f"../datasets/adv_{level}/test/{i}")

# Model

In [5]:
# 3x3 convolution
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,
                     stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(3, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(8)
        self.fc = nn.Linear(64, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Loading Data

In [6]:
# Batch size
batch_size = 100

# Normalize
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

# Transform
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor(),
])

# CIFAR-10 Dataset
train_dataset = torchvision.datasets.CIFAR10(
    root='../datasets/',
    train=True,
    transform=transform,
    download=True
)

test_dataset = torchvision.datasets.CIFAR10(
    root='../datasets/',
    train=False,
    transform=transform,
    download=True
)

# Data loader
train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=False
)

test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=False
)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
class_names = [
    "airplane",
    "automobile",
    "bird",
    "cat",
    "deer",
    "dog",
    "frog",
    "horse",
    "ship",
    "truck",
]

# Save CIFAR10 as Image

In [8]:
prepare_folders(0)

In [9]:
def cifar10_to_png(is_train_data=True):
    data_type, loader = ("train", train_loader) if is_train_data else ("test", test_loader)

    for i, (images, labels) in enumerate(loader):
        print(f"[{i*batch_size}/{len(loader)*batch_size}]")

        # Save images
        for j, (image, label) in enumerate(zip(images, labels)):
            save_image(image, f"../datasets/adv_0/{data_type}/{label}/{i * batch_size + j}.png")

In [10]:
cifar10_to_png(True)

[0/50000]
[100/50000]
[200/50000]
[300/50000]
[400/50000]
[500/50000]
[600/50000]
[700/50000]
[800/50000]
[900/50000]
[1000/50000]
[1100/50000]
[1200/50000]
[1300/50000]
[1400/50000]
[1500/50000]
[1600/50000]
[1700/50000]
[1800/50000]
[1900/50000]
[2000/50000]
[2100/50000]
[2200/50000]
[2300/50000]
[2400/50000]
[2500/50000]
[2600/50000]
[2700/50000]
[2800/50000]
[2900/50000]
[3000/50000]
[3100/50000]
[3200/50000]
[3300/50000]
[3400/50000]
[3500/50000]
[3600/50000]
[3700/50000]
[3800/50000]
[3900/50000]
[4000/50000]
[4100/50000]
[4200/50000]
[4300/50000]
[4400/50000]
[4500/50000]
[4600/50000]
[4700/50000]
[4800/50000]
[4900/50000]
[5000/50000]
[5100/50000]
[5200/50000]
[5300/50000]
[5400/50000]
[5500/50000]
[5600/50000]
[5700/50000]
[5800/50000]
[5900/50000]
[6000/50000]
[6100/50000]
[6200/50000]
[6300/50000]
[6400/50000]
[6500/50000]
[6600/50000]
[6700/50000]
[6800/50000]
[6900/50000]
[7000/50000]
[7100/50000]
[7200/50000]
[7300/50000]
[7400/50000]
[7500/50000]
[7600/50000]
[7700/50000

In [11]:
cifar10_to_png(False)

[0/10000]
[100/10000]
[200/10000]
[300/10000]
[400/10000]
[500/10000]
[600/10000]
[700/10000]
[800/10000]
[900/10000]
[1000/10000]
[1100/10000]
[1200/10000]
[1300/10000]
[1400/10000]
[1500/10000]
[1600/10000]
[1700/10000]
[1800/10000]
[1900/10000]
[2000/10000]
[2100/10000]
[2200/10000]
[2300/10000]
[2400/10000]
[2500/10000]
[2600/10000]
[2700/10000]
[2800/10000]
[2900/10000]
[3000/10000]
[3100/10000]
[3200/10000]
[3300/10000]
[3400/10000]
[3500/10000]
[3600/10000]
[3700/10000]
[3800/10000]
[3900/10000]
[4000/10000]
[4100/10000]
[4200/10000]
[4300/10000]
[4400/10000]
[4500/10000]
[4600/10000]
[4700/10000]
[4800/10000]
[4900/10000]
[5000/10000]
[5100/10000]
[5200/10000]
[5300/10000]
[5400/10000]
[5500/10000]
[5600/10000]
[5700/10000]
[5800/10000]
[5900/10000]
[6000/10000]
[6100/10000]
[6200/10000]
[6300/10000]
[6400/10000]
[6500/10000]
[6600/10000]
[6700/10000]
[6800/10000]
[6900/10000]
[7000/10000]
[7100/10000]
[7200/10000]
[7300/10000]
[7400/10000]
[7500/10000]
[7600/10000]
[7700/10000

# ReColorAdv Attack Model

In [11]:
_, normalizer = load_pretrained_cifar10_model('pretrained_models/normal.resnet32.pt')
model = torch.load('pretrained_models/resnet_def_10_epochs')

if utils.use_gpu():
    model.cuda()

In [12]:
# This threat model defines the regularization parameters of the attack.
recoloradv_threat = ap.ThreatModel(pt.ReColorAdv, {
    'xform_class': ct.FullSpatial, 
    'cspace': cs.CIELUVColorSpace(), # controls the color space used
    'lp_style': 'inf',
    'lp_bound': [0.06, 0.06, 0.06],  # [epsilon_1, epsilon_2, epsilon_3]
    'xform_params': {
      'resolution_x': 16,            # R_1
      'resolution_y': 32,            # R_2
      'resolution_z': 32,            # R_3
    },
    'use_smooth_loss': True,
})


# Now, we define the main optimization term (the Carlini & Wagner f6 loss).
adv_loss = lf.CWLossF6(model, normalizer)

# We also need the smoothness loss.
smooth_loss = lf.PerturbationNormLoss(lp=2)

# We combine them with a RegularizedLoss object.
attack_loss = lf.RegularizedLoss({'adv': adv_loss, 'smooth': smooth_loss}, 
                                 {'adv': 1.0,      'smooth': 0.05},   # lambda = 0.05
                                 negate=True) # Need this true for PGD type attacks

# PGD is used to optimize the above loss.
pgd_attack_obj = aa.PGD(model, normalizer, recoloradv_threat, attack_loss)

# Generating Adversarial Sample

In [13]:
prepare_folders("iter")

In [14]:
def generate_adversarial_sample(is_train_data=True):
    data_type, loader = ("train", train_loader) if is_train_data else ("test", test_loader)

    for i, (images, labels) in enumerate(loader):
        print(f"[{i*batch_size}/{len(loader)*batch_size}]")

        if utils.use_gpu():
            images = images.cuda()
            labels = labels.cuda()

        # We run the attack for 10 iterations at learning rate 0.01.
        with HidePrint():
            perturbation = pgd_attack_obj.attack(images, labels, num_iterations=10, signed=False, 
                                                 optimizer=optim.Adam, optimizer_kwargs={'lr': 0.01},
                                                 verbose=True)
        
        # Now, we can collect the successful adversarial examples and display them.
        successful_advs, successful_origs, idxs = perturbation.my_collect_successful(model, normalizer)

        # Save images
        for j, (image, label) in enumerate(zip(successful_advs, labels)):
            save_image(image, f"../datasets/adv_iter/{data_type}/{label}/{i * batch_size + j}.png")

In [16]:
generate_adversarial_sample(True)

[0/50000]
[100/50000]
[200/50000]
[300/50000]
[400/50000]
[500/50000]
[600/50000]
[700/50000]
[800/50000]
[900/50000]
[1000/50000]
[1100/50000]
[1200/50000]
[1300/50000]
[1400/50000]
[1500/50000]
[1600/50000]
[1700/50000]
[1800/50000]
[1900/50000]
[2000/50000]
[2100/50000]
[2200/50000]
[2300/50000]
[2400/50000]
[2500/50000]
[2600/50000]
[2700/50000]
[2800/50000]
[2900/50000]
[3000/50000]
[3100/50000]
[3200/50000]
[3300/50000]
[3400/50000]
[3500/50000]
[3600/50000]
[3700/50000]
[3800/50000]
[3900/50000]
[4000/50000]
[4100/50000]
[4200/50000]
[4300/50000]
[4400/50000]
[4500/50000]
[4600/50000]
[4700/50000]
[4800/50000]
[4900/50000]
[5000/50000]
[5100/50000]
[5200/50000]
[5300/50000]
[5400/50000]
[5500/50000]
[5600/50000]
[5700/50000]
[5800/50000]
[5900/50000]
[6000/50000]
[6100/50000]
[6200/50000]
[6300/50000]
[6400/50000]
[6500/50000]
[6600/50000]
[6700/50000]
[6800/50000]
[6900/50000]
[7000/50000]
[7100/50000]
[7200/50000]
[7300/50000]
[7400/50000]
[7500/50000]
[7600/50000]
[7700/50000

In [15]:
generate_adversarial_sample(False)

[0/10000]
[100/10000]
[200/10000]
[300/10000]
[400/10000]
[500/10000]
[600/10000]
[700/10000]
[800/10000]
[900/10000]
[1000/10000]
[1100/10000]
[1200/10000]
[1300/10000]
[1400/10000]
[1500/10000]
[1600/10000]
[1700/10000]
[1800/10000]
[1900/10000]
[2000/10000]
[2100/10000]
[2200/10000]
[2300/10000]
[2400/10000]
[2500/10000]
[2600/10000]
[2700/10000]
[2800/10000]
[2900/10000]
[3000/10000]
[3100/10000]
[3200/10000]
[3300/10000]
[3400/10000]
[3500/10000]
[3600/10000]
[3700/10000]
[3800/10000]
[3900/10000]
[4000/10000]
[4100/10000]
[4200/10000]
[4300/10000]
[4400/10000]
[4500/10000]
[4600/10000]
[4700/10000]
[4800/10000]
[4900/10000]
[5000/10000]
[5100/10000]
[5200/10000]
[5300/10000]
[5400/10000]
[5500/10000]
[5600/10000]
[5700/10000]
[5800/10000]
[5900/10000]
[6000/10000]
[6100/10000]
[6200/10000]
[6300/10000]
[6400/10000]
[6500/10000]
[6600/10000]
[6700/10000]
[6800/10000]
[6900/10000]
[7000/10000]
[7100/10000]
[7200/10000]
[7300/10000]
[7400/10000]
[7500/10000]
[7600/10000]
[7700/10000

# Evaluation

In [18]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
# Load test data from adversarial sample
transform_norm = transforms.Compose([
    transforms.Pad(4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor(),
    normalize
])

test_dataset_adv = torchvision.datasets.ImageFolder(root='../datasets/adv_1/test', transform=transform_norm)

test_loader_adv = torch.utils.data.DataLoader(
    dataset=test_dataset_adv,
    batch_size=batch_size,
    shuffle=False
)

In [20]:
def test(model, test_loader):
    model.eval()
    
    predictions = []
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            prediction = torch.max(model(images), 1)[1].data.squeeze()
            predictions += (prediction.tolist())
            correct += (prediction == labels).sum().item()
            total += len(labels)
    
    return correct / total, predictions

In [21]:
test(model, test_loader_adv)[0]

0.3609